In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from transformers import CLIPProcessor, CLIPModel, TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
import matplotlib.pyplot as plt
import cv2
import torch
import numpy as np
import supervision as sv

from pytesseract import Output
import pytesseract

import cv2
import numpy as np
import pytesseract
from scipy.ndimage import rotate

In [ ]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

text_processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
text_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

In [ ]:
def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    return image

def get_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for i in range(len(sorted_anns)):
        ann = sorted_anns[i]
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    return img

def show_annotated(annotated_image):
    plt.figure(figsize=(20,20))
    plt.imshow(annotated_image)
    # masked_img = show_anns(masks)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    # ax.imshow(masked_img[0:100])
    plt.axis('off')
    plt.show()

In [ ]:
def text_preprocess(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    return thresh

def text_orientation_correction(image):
    edges = cv2.Canny(image, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=100, maxLineGap=10)
    angles = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        angles.append(angle)
    average_angle = np.mean(angles)
    corrected_image = rotate(image, -average_angle)
    return corrected_image

def extract_text(image):
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(image, config=custom_config)
    return text

In [ ]:
def classify_mask_as_book(masked_img, labels=["book", "not book"]):
    # Apply the mask to the image
    # masked_img = np.zeros_like(image)
    # masked_img[mask] = image[mask]

    # Prepare the image for the CLIP model
    pil_image = Image.fromarray(masked_img)
    inputs = processor(text=labels, images=pil_image, return_tensors="pt", padding=True)
    outputs = clip_model(**inputs)

    # Get the predicted label
    logits_per_image = outputs.logits_per_image
    probs = torch.softmax(logits_per_image, dim=1)
    best_label_idx = torch.argmax(probs)
    return labels[best_label_idx] == "book"

def extract_text_from_mask(masked_img):
    # Apply the mask to the image
    # masked_img = np.zeros_like(image)
    # masked_img[mask] = image[mask]

    # Extract text using pytesseract
    text = pytesseract.image_to_string(masked_img)
    return text.strip()

In [ ]:
image = load_image("original_image.jpg")
sam = sam_model_registry["vit_h"](checkpoint="sam_vit_h_4b8939.pth")
sam.to(device="cpu")
image = cv2.resize(image, (1024, 1024), interpolation=cv2.INTER_LINEAR)
print(1)

mask_generator = SamAutomaticMaskGenerator(model=sam,
                                           points_per_side=32,
                                           points_per_batch=128,
                                           pred_iou_thresh=0.98,
                                           stability_score_thresh=0.97,
                                           crop_n_layers=1,
                                           crop_n_points_downscale_factor=2,
                                           min_mask_region_area=500)

In [ ]:
masks = mask_generator.generate(image)

In [ ]:
mask_annotator = sv.MaskAnnotator(color_lookup = sv.ColorLookup.INDEX)
detections = sv.Detections.from_sam(masks)
annotated_image = mask_annotator.annotate(image, detections)

In [ ]:
show_annotated(annotated_image)

In [ ]:
book_count = 0
count = 0
for i, mask_dict in enumerate(masks):
    mask = np.repeat(mask_dict['segmentation'][:, :, np.newaxis].astype(np.uint8), 3, axis=2) * 255
    masked_img = image.copy()
    cv2.bitwise_and(image.astype(np.uint8), mask, masked_img)
    count += 1

    if classify_mask_as_book(masked_img):
        book_count += 1
        title = extract_text_from_mask(masked_img)
        print(f"Book {book_count} title: {title}")
    else:
        print(f"Segment {count} not a book")